In [11]:
import pandas as pd
import glob
import os

In [12]:
KENPOM_DATA_DIR = "/Users/jameswhedbee/jupyter_notebooks/DATA/MarchMadness/KenPom"
KAGGLE_DATA_DIR = "/Users/jameswhedbee/jupyter_notebooks/DATA/MarchMadness/MDataFiles_Stage2"
OUTPUT_DATA_DIR = "/Users/jameswhedbee/jupyter_notebooks/DATA/MarchMadness/output"

In [13]:
KENPOM_COLS = ["Season", "TeamName", "AdjOE", "AdjDE"]

In [14]:
all_files = glob.glob(os.path.join(KENPOM_DATA_DIR, "*.csv"))

data = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    data.append(df)

kenpom_df = pd.concat(data, axis=0, ignore_index=True)[KENPOM_COLS]
kenpom_df["LowerTeamName"] = kenpom_df["TeamName"].str.lower()
kenpom_df

,Season,TeamName,AdjOE,AdjDE,LowerTeamName
0,2013,Air Force,112.8840,104.4400,air force
1,2013,Akron,107.8010,94.0583,akron
2,2013,Alabama,104.1100,93.3046,alabama
3,2013,Alabama A&M,90.5796,111.4980,alabama a&m
4,2013,Alabama St.,92.2111,112.0520,alabama st.
...,...,...,...,...,...
6182,2004,Wright St.,103.6850,104.8470,wright st.
6183,2004,Wyoming,104.4840,99.8136,wyoming
6184,2004,Xavier,110.7610,92.9239,xavier
6185,2004,Yale,99.3150,103.3250,yale


In [15]:
teams_df = pd.read_csv(f"{KAGGLE_DATA_DIR}/MTeams.csv")
spellings_df = pd.read_csv(f"{KAGGLE_DATA_DIR}/MTeamSpellings.csv")
spellings_df[1040:1090]

,TeamNameSpelling,TeamID
1040,ut-arlington,1426
1041,ut-martin,1404
1042,utah,1428
1043,utah st,1429
1044,utah st.,1429
1045,utah state,1429
1046,utah valley,1430
1047,utah valley st.,1430
1048,utah-st,1429
1049,utah-state,1429


In [16]:
merged_df = kenpom_df.merge(spellings_df, left_on="LowerTeamName", right_on="TeamNameSpelling")[KENPOM_COLS + ["TeamID"]]
merged_df

,Season,TeamName,AdjOE,AdjDE,TeamID
0,2013,Air Force,112.884,104.4400,1102
1,2011,Air Force,105.717,101.1380,1102
2,2003,Air Force,105.340,103.7220,1102
3,2019,Air Force,100.431,107.2750,1102
4,2007,Air Force,116.286,97.0562,1102
...,...,...,...,...,...
5912,2017,Fort Wayne,109.530,108.0540,1236
5913,2018,Fort Wayne,107.616,106.4470,1236
5914,2021,Bellarmine,107.357,108.6070,1468
5915,2021,Merrimack,91.086,101.3770,1467


In [17]:
missing_teams = set(kenpom_df["TeamName"]) - set(merged_df["TeamName"])
missing_teams

{'Arkansas Little Rock',
 'Arkansas Pine Bluff',
 'Bethune Cookman',
 'Cal St. Bakersfield',
 'Dixie St.',
 'Illinois Chicago',
 'LIU',
 'Louisiana Lafayette',
 'Louisiana Monroe',
 'Maryland Eastern Shore',
 'Mississippi Valley St.',
 'Southeast Missouri St.',
 'Southwest Missouri St.',
 'Southwest Texas St.',
 'St. Francis NY',
 'St. Francis PA',
 'Tarleton St.',
 'Tennessee Martin',
 'Texas A&M Corpus Chris',
 'Texas Pan American',
 'UT Rio Grande Valley',
 'Winston Salem St.'}

In [18]:
missing_df = kenpom_df[kenpom_df["TeamName"].isin(missing_teams)].set_index("TeamName")
missing_df["LowerTeamName"] = missing_df["LowerTeamName"].str.replace(" ", "-").str.replace("\.", "").str.replace("-st", "-state")

# Manual overrides
missing_df.at["Dixie St.", "LowerTeamName"] = "dixie st"
missing_df.at["LIU", "LowerTeamName"] = "liu-brooklyn"
missing_df.at["Southwest Missouri St.", "LowerTeamName"] = "southwest missouri state"
missing_df.at["Southwest Texas St.", "LowerTeamName"] = "texas-st"
missing_df.at["Texas A&M Corpus Chris", "LowerTeamName"] = "texas a&m cc"
missing_df.at["Tarleton St.", "LowerTeamName"] = "tarleton st"
missing_df.at["UT Rio Grande Valley", "LowerTeamName"] = "utrgv"


dash_merge_df = missing_df.reset_index().merge(spellings_df, left_on="LowerTeamName", right_on="TeamNameSpelling")[KENPOM_COLS + ["TeamID"]]
merged_df = pd.concat([merged_df, dash_merge_df])
missing_teams = set(kenpom_df["TeamName"]) - set(merged_df["TeamName"])
missing_teams

<ipython-input-18-0b009b6d05a8>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  missing_df["LowerTeamName"] = missing_df["LowerTeamName"].str.replace(" ", "-").str.replace("\.", "").str.replace("-st", "-state")


set()

In [19]:
merged_df.to_csv(f"{OUTPUT_DATA_DIR}/kenpom.csv", index=False)

In [20]:
merged_df.isna().sum()

Season      0
TeamName    0
AdjOE       0
AdjDE       0
TeamID      0
dtype: int64